In [ ]:
!pip install -U sentence-transformers --quiet
!pip install pytorch-lightning==1.1.8 --quiet

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib.request, zipfile, os
import time
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.model_selection import StratifiedKFold
import pickle, gc

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

In [3]:
mkdir data

## Download data

In [4]:
if not os.path.exists('./data/attack_annotations.tsv'):
  file_path = 'data/4054689.zip'
  urllib.request.urlretrieve('https://ndownloader.figshare.com/articles/4054689/versions/6', file_path)
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
      zip_ref.extractall('data')

  file_path = 'data/4267550.zip'
  urllib.request.urlretrieve('https://ndownloader.figshare.com/articles/4267550/versions/5', file_path)
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
      zip_ref.extractall('data')

  file_path = 'data/4563973.zip'
  urllib.request.urlretrieve('https://ndownloader.figshare.com/articles/4563973/versions/2', file_path)
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
      zip_ref.extractall('data')

In [5]:
aggression_data = pd.read_csv('./data/aggression_annotated_comments.tsv', sep='\t')
aggression_annotations = pd.read_csv('./data/aggression_annotations.tsv', sep='\t')
aggression_worker_demographics = pd.read_csv('/content/data/aggression_worker_demographics.tsv', sep='\t')

In [6]:
aggression_annotations = aggression_annotations.merge(aggression_worker_demographics)

## Create text and annotator one hot feature vectors

In [8]:
aggression_text_features = aggression_data.loc[:, ['year', 'logged_in', 'ns', 'sample']].fillna('empty')

year_onehot = pd.get_dummies(aggression_text_features.year).values
logged_in_onehot = pd.get_dummies(aggression_text_features.logged_in).values
ns_onehot = pd.get_dummies(aggression_text_features.ns).values
sample_onehot = pd.get_dummies(aggression_text_features['sample']).values

text_features = np.hstack([year_onehot, logged_in_onehot, ns_onehot, sample_onehot])

In [9]:
aggression_worker_demographics = aggression_worker_demographics.fillna('empty')

worker_id_onehot = pd.get_dummies(aggression_worker_demographics.worker_id).values
gender_onehot = pd.get_dummies(aggression_worker_demographics.gender).values
english_first_language_onehot = pd.get_dummies(aggression_worker_demographics.english_first_language).values
age_group_onehot = pd.get_dummies(aggression_worker_demographics.age_group).values
education_onehot = pd.get_dummies(aggression_worker_demographics.education).values

annotator_features = np.hstack([gender_onehot, english_first_language_onehot, age_group_onehot, education_onehot])

## Bert embeddings

In [10]:
rev_ids = aggression_data.rev_id.to_list()
comments = aggression_data.comment.to_list()
comments = [c.replace('NEWLINE_TOKEN', ' ') for c in comments]

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased")
model = model.to(device)

In [12]:
from tqdm import tqdm

def get_embeddings(max_seq_len=256):
  def batch(iterable, n=1):
      l = len(iterable)
      for ndx in range(0, l, n):
          yield iterable[ndx:min(ndx + n, l)]

  all_embeddings = []
  for b_comments in tqdm(batch(comments, 200), total=len(comments)/200):
    
    with torch.no_grad():
      batch_encoding = tokenizer.batch_encode_plus(
            b_comments,
            padding='longest',
            add_special_tokens=True,
            truncation=True, max_length=max_seq_len,
            return_tensors='pt',
        ).to(device)

      emb = model(**batch_encoding)

    for i in range(emb[0].size()[0]):
      all_embeddings.append(emb[0][i, batch_encoding['input_ids'][i] > 0, :].mean(axis=0)[None, :])

  return all_embeddings

In [13]:
all_embeddings = get_embeddings(256)
all_embeddings = torch.cat(all_embeddings, axis=0)

580it [39:00,  4.03s/it]


## Training

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self, classes_num=2, feature_num=768):
        super(Net, self).__init__()
        self.feature_num = feature_num
        self.fc1 = nn.Linear(feature_num, classes_num) 

    def forward(self, x, text_lengths=None):

        x = x.view(-1, self.feature_num)
        x = self.fc1(x)
        return x

In [15]:
import torch.utils.data as data

class BatchIndexedDataset(data.Dataset):
    def __init__(self, X, y, embeddings):
        self.X = X
        self.y = torch.tensor(y).long()
        self.embeddings = embeddings

        self.aggression_text_features = torch.tensor(text_features).to(device)
        self.worker_id_onehot = torch.tensor(worker_id_onehot).to(device)
        self.annotator_features = torch.tensor(annotator_features).to(device)

    def __getitem__(self, index):
        revs_X = self.X[index, 0]
        workers_X = self.X[index, 1]

        batch_X = self.embeddings[revs_X]
        batch_y = self.y[index]

        if CFG['scenario'] == 's2':
          batch_X = torch.cat([batch_X, self.annotator_features[workers_X], self.aggression_text_features[revs_X]], dim=1)

        elif CFG['scenario'] == 's3':
          batch_X = torch.cat([batch_X, self.annotator_features[workers_X], self.aggression_text_features[revs_X], self.worker_id_onehot[workers_X]], dim=1)

        elif CFG['scenario'] == 's4':
          negative_embeddings = annotator_negative_embeddings[workers_X].to(device)
          positive_embeddings = annotator_positive_embeddings[workers_X].to(device)
          batch_X = torch.cat([batch_X, self.annotator_features[workers_X], self.aggression_text_features[revs_X], negative_embeddings, positive_embeddings], dim=1)

        return batch_X.float().to(device), batch_y.to(device)
    
    def __len__(self):
        return len(self.y)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader, WeightedRandomSampler
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning import loggers as pl_loggers

def prepare_dataloader(X, y):
  dataset = BatchIndexedDataset(X, y, all_embeddings)        
  sampler = data.sampler.BatchSampler(
      data.sampler.RandomSampler(dataset),
      batch_size=CFG['batch_size'],
      drop_last=False)
  
  return data.DataLoader(dataset, sampler=sampler)

def evaluate(train_X, dev_X, test_X, train_y, dev_y, test_y):
    """ Train classifier """
    train_loader = prepare_dataloader(train_X, train_y)
    val_loader = prepare_dataloader(dev_X, dev_y)
    test_loader = prepare_dataloader(test_X, test_y)

    feature_num = next(iter(val_loader))[0].size(2)
    model = HateClassifier(2, feature_num=feature_num).to(device)

    tb_logger = pl_loggers.TensorBoardLogger('logs/')
    trainer = pl.Trainer(gpus=1 if torch.cuda.is_available() else 0, max_epochs=CFG['epochs'], progress_bar_refresh_rate=20)
    trainer.fit(model, train_loader, val_loader)

    model.eval()
    model = model.to(device)
    
    test_probabs = [] 
    true_labels = []
    with torch.no_grad():
      for batch_text_X, batch_text_y in test_loader:
        test_probabs.append(model(batch_text_X.to(device)))
        true_labels.extend(batch_text_y.to(device).flatten().tolist())

    test_probabs = torch.cat(test_probabs, dim=0)
    test_predictions  = test_probabs.argmax(dim=1)

    y_true = np.array(true_labels).flatten()
    y_pred = test_predictions.tolist() 

    print(classification_report(y_true, y_pred))
    result_dict = classification_report(y_true, y_pred, output_dict=True)
    
    return result_dict

class HateClassifier(pl.LightningModule):
    def __init__(self, classes_num=2, feature_num=768):
        super().__init__()
        self.model = Net(classes_num=classes_num, feature_num=feature_num).to(device)

        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.train_f1 = pl.metrics.F1(1,average=None)
        self.valid_f1 = pl.metrics.F1(1, average=None)
        self.valid_conf = pl.metrics.ConfusionMatrix(2)

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.flatten()
        output = self.forward(x)

        loss = nn.CrossEntropyLoss(torch.tensor(CFG['class_weights']).to(device))(output, y)
        self.log('train_loss',  loss, on_epoch=True)
        self.log('train_acc', self.train_acc(output, y), prog_bar=True)
        self.log('train_f1', self.train_f1(output, y), prog_bar=True)

        return loss

    def training_epoch_end(self, outs):
        epoch_acc = self.train_acc.compute()
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.flatten()
        output = self.forward(x)
        loss = nn.CrossEntropyLoss(torch.tensor(CFG['class_weights']).to(device))(output, y)

        self.log('valid_loss', loss)
        self.log('valid_acc', self.valid_acc(output, y), prog_bar=True)
        self.log('valid_f1', self.valid_f1(output, y), prog_bar=True)
        self.log('valid_conf', self.valid_conf(output, y))
        
        return {'loss': loss, 'true_labels': output, 'predictions': y}

    def validation_epoch_end(self, outs):
        val_epoch_acc = self.valid_acc.compute()
        self.valid_f1.compute()
        self.valid_conf.compute()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=CFG['lr'])
        return optimizer

## Annotator personal embeddings

In [17]:
rev_id_idx_dict = aggression_data.loc[:, ['rev_id']].reset_index().set_index('rev_id').to_dict()['index']
worker_id_idx_dict = aggression_worker_demographics.loc[:, ['worker_id']].reset_index().set_index('worker_id').to_dict()['index']

In [18]:
train_X = aggression_annotations.loc[aggression_annotations.rev_id.isin(aggression_data[aggression_data.split == 'train'].rev_id.values)].loc[:, ['rev_id', 'worker_id']]
dev_X = aggression_annotations.loc[aggression_annotations.rev_id.isin(aggression_data[aggression_data.split == 'dev'].rev_id.values)].loc[:, ['rev_id', 'worker_id']]
test_X = aggression_annotations.loc[aggression_annotations.rev_id.isin(aggression_data[aggression_data.split == 'test'].rev_id.values)].loc[:, ['rev_id', 'worker_id']]

train_y = aggression_annotations.loc[aggression_annotations.rev_id.isin(aggression_data[aggression_data.split == 'train'].rev_id.values)].aggression
dev_y = aggression_annotations.loc[aggression_annotations.rev_id.isin(aggression_data[aggression_data.split == 'dev'].rev_id.values)].aggression
test_y = aggression_annotations.loc[aggression_annotations.rev_id.isin(aggression_data[aggression_data.split == 'test'].rev_id.values)].aggression

for df in [train_X, dev_X, test_X]:
  df['worker_id'] = df['worker_id'].apply(lambda w_id: worker_id_idx_dict[w_id])
  df['rev_id'] = df['rev_id'].apply(lambda r_id: rev_id_idx_dict[r_id])

train_X, dev_X, test_X, train_y, dev_y, test_y = train_X.values, dev_X.values, test_X.values, train_y.values, dev_y.values, test_y.values

In [19]:
train_rev_ids = aggression_data[aggression_data.split == 'train'].rev_id.to_list()

In [20]:
annotator_negative_embeddings = torch.zeros(len(worker_id_idx_dict.keys()), 768)
annotator_positive_embeddings = torch.zeros(len(worker_id_idx_dict.keys()), 768)

worker_annotations = aggression_annotations[aggression_annotations.rev_id.isin(train_rev_ids)].groupby(['worker_id', 'aggression'])['rev_id'].apply(list).to_dict()

In [21]:
for i in worker_id_idx_dict.keys():
  if (i, 0.0) in worker_annotations:
    negative_text_idxs = [rev_id_idx_dict[r_idx] for r_idx in worker_annotations[(i, 0.0)]]
    annotator_negative_embeddings[worker_id_idx_dict[i]] = all_embeddings[negative_text_idxs].mean(axis=0)
  if (i, 1.0) in worker_annotations:
    positive_text_idxs = [rev_id_idx_dict[r_idx] for r_idx in worker_annotations[(i, 1.0)]]
    annotator_positive_embeddings[worker_id_idx_dict[i]] = all_embeddings[positive_text_idxs].mean(axis=0)

## S1

In [22]:
CFG = {
    'lr': 7*1e-4, 
    'epochs': 20,
    'class_weights': [1.0, 1.0],
    'batch_size': 1000,
    'scenario': 's1'
}

In [ ]:
results_s1 = {}
for i in range(10):
  results_s1[i] = evaluate(train_X, dev_X, test_X, train_y, dev_y, test_y)

## S2

In [ ]:
CFG = {
    #'lr': 7*1e-4, 
    'lr': 7*1e-4, 
    'epochs': 20,
    'class_weights': [1.0, 1.0],
    'batch_size': 1000,
    'scenario': 's2'
}

In [ ]:
results_s2 = {}
for i in range(10):
  results_s2[i] = evaluate(train_X, dev_X, test_X, train_y, dev_y, test_y)

## S3

In [ ]:
CFG = {
    #'lr': 7*1e-4, 
    'lr': 7*1e-4, 
    'epochs': 20,
    'class_weights': [1.0, 1.0],
    'batch_size': 1000,
    'scenario': 's3'
}

In [ ]:
results_s3 = {}
for i in range(10):
  results_s3[i] = evaluate(train_X, dev_X, test_X, train_y, dev_y, test_y)

## S4

In [25]:
CFG = {
    #'lr': 7*1e-4, 
    'lr': 7*1e-4, 
    'epochs': 20,
    'class_weights': [1.0, 1.0],
    'batch_size': 1000,
    'scenario': 's4'
}

In [ ]:
results_s4 = {}
for i in range(10):
  results_s4[i] = evaluate(train_X, dev_X, test_X, train_y, dev_y, test_y)

## Results

In [ ]:
def get_mean_results(results):
  accuracy = np.mean([results[i]['accuracy'] for i in results.keys()])
  precision_macro = np.mean([results[i]['macro avg']['precision'] for i in results.keys()])
  recall_macro = np.mean([results[i]['macro avg']['recall'] for i in results.keys()])
  f1_macro = np.mean([results[i]['macro avg']['f1-score'] for i in results.keys()])
  precision_a = np.mean([results[i]['1']['precision'] for i in results.keys()])
  recall_a = np.mean([results[i]['1']['recall'] for i in results.keys()])
  f1_a = np.mean([results[i]['1']['f1-score'] for i in results.keys()])

  return {'accuracy': accuracy, 
          'precision_macro': precision_macro,
          'recall_macro': recall_macro,
          'f1_macro': f1_macro,
          'precision_a': precision_a,
          'recall_a': recall_a,
          'f1_a': f1_a,
          }

print('S1')
print(get_mean_results(results_s1))

print('S2')
print(get_mean_results(results_s2))

print('S3')
print(get_mean_results(results_s3))

print('S4')
print(get_mean_results(results_s4))